<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/applications/mangrove_class/03_Mangrove_Class_Percentiles_DEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The file path on google drive for ROI output vector file.
vec_cls_roi_file = "/content/drive/MyDrive/mangrove_chng_cls/n0940e0190_tile.geojson"

# Start and End date
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)

# Output no data value
no_data_val = 0.0


In [ ]:
# Specify the paths to the vector files with the training points for each of
# classes. Note these files include samples for a larger areas than defined
# in the roi file.
vec_mng_smpls_file = '/content/drive/MyDrive/mangrove_chng_cls/mng_smpls.geojson'
vec_wtr_smpls_file = '/content/drive/MyDrive/mangrove_chng_cls/wtr_smpls.geojson'
vec_oth_smpls_file = '/content/drive/MyDrive/mangrove_chng_cls/oth_smpls.geojson'

In [ ]:
# The following function calls convert the training sample points to GEE
# point geometries. In addition, the points a clipped to the ROI for the
# region being classified and subsampled to reduce the number of points
# used to train the classifier as this reduced the memory footprint of
# the notebook which is limited.
gee_mng_pts = pb_gee_tools.convert_types.get_gee_pts(vec_mng_smpls_file, rnd_smpl = 50, rnd_seed = 42, vec_roi_file=vec_cls_roi_file)
gee_wtr_pts = pb_gee_tools.convert_types.get_gee_pts(vec_wtr_smpls_file, rnd_smpl = 50, rnd_seed = 42, vec_roi_file=vec_cls_roi_file)
gee_oth_pts = pb_gee_tools.convert_types.get_gee_pts(vec_oth_smpls_file, rnd_smpl = 50, rnd_seed = 42, vec_roi_file=vec_cls_roi_file)

In [ ]:
# Merge the training samples into a single Feature Collection with a variable
# for the ID of each class.
train_smpls = ee.FeatureCollection([
    ee.Feature(gee_mng_pts, {'class': 1}),
    ee.Feature(gee_wtr_pts, {'class': 2}),
    ee.Feature(gee_oth_pts, {'class': 3}),
])

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_cls_roi_gdf = geopandas.read_file(vec_cls_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_cls_roi_bbox = vec_cls_roi_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_cls_west = gp_cls_roi_bbox[0]
roi_cls_east = gp_cls_roi_bbox[2]
roi_cls_north = gp_cls_roi_bbox[3]
roi_cls_south = gp_cls_roi_bbox[1]
cls_tile_aoi = ee.Geometry.BBox(roi_cls_west, roi_cls_south, roi_cls_east, roi_cls_north)

cls_cnt_pt_x = roi_cls_west + (roi_cls_east - roi_cls_west) / 2
cls_cnt_pt_y = roi_cls_south + (roi_cls_north - roi_cls_south) / 2
cls_cnt_pt = ee.Geometry.Point([cls_cnt_pt_x, cls_cnt_pt_y])

In [ ]:
# Load the Sentinel-2 imagery.
s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
    aoi=cls_tile_aoi,
    start_date=start_date,
    end_date=end_date,
    cloud_thres = 70,
)

# Create a median composite clipped to the ROI.
s2_med_image = s2_img_col.median().clip(cls_tile_aoi).unmask(no_data_val)

In [ ]:
# A function which will be used to calculate a number of image band indices
# for each input image.
def calc_band_indices(img):
    img = img.multiply(.0001).float()
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    ndwi = img.normalizedDifference(["B8", "B11"]).rename("NDWI")
    nbr = img.normalizedDifference(["B8", "B12"]).rename("NBR")
    evi = img.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
      {
          'NIR': img.select('B8'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2'),
      },
    ).rename(['EVI'])
    mvi = img.expression(
      '((NIR - GREEN) / (SWIR - GREEN))',
      {
          'NIR': img.select('B8'),
          'SWIR': img.select('B11'),
          'GREEN': img.select('B3'),
      },
    ).rename(['MVI'])
    remi = img.expression(
      '((REDEDGE - RED) / (SWIR + GREEN))',
      {
          'REDEDGE': img.select('B6'),
          'RED': img.select('B4'),
          'SWIR': img.select('B11'),
          'GREEN': img.select('B3'),
      },
    ).rename(['REMI'])
    return img.addBands([ndvi, ndwi, nbr, evi, mvi, remi])

In [ ]:
# Map the input images to calculate the indices for
# each the input images.
s2_indices = s2_img_col.map(calc_band_indices)

In [ ]:
# List the bands in each image
bands = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12", "NDVI", "NDWI", "NBR", "EVI", "MVI", "REMI"]

In [ ]:
# Loop through the input bands to calculate the 10th, 50th (median) and 90th
# percentiles for the reflectance bands and indices.
s2_img_percents = []
for band in bands:
  s2_img_percent = s2_indices.select(band).reduce(ee.Reducer.percentile([10, 50, 90]))
  s2_img_percents.append(s2_img_percent)

In [ ]:
# Convert the list of percentile images to a single image
s2_img_percent = ee.ImageCollection.fromImages(s2_img_percents).toBands()
s2_img_percent

In [ ]:
# Import the elevation dataset.
dem_img = ee.ImageCollection('COPERNICUS/DEM/GLO30').select(["DEM"]).mean()

# Add the elevation data to the s2_img_percent.
s2_img_percent_dem = s2_img_percent.addBands(dem_img)

In [ ]:
map_obj = geemap.Map()

# Specify the centre point and scale of the map
map_obj.centerObject(cls_cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {"min": [150, 100, 187], "max": [4136, 4000, 2712], "bands": ["B8", "B11", "B4"]}

# Add the Sentinel-2 Composite
map_obj.addLayer(s2_med_image, vis_params, "Sentinel-2")

map_obj

In [ ]:
# Get a list of the bands names.
img_bands = s2_img_percent_dem.bandNames()
img_bands

In [ ]:
# Extract the training samples fromt the input imagery.
train_data = s2_img_percent_dem.select(img_bands).sampleRegions(
    collection=train_smpls, properties=["class"], scale=10
)

In [ ]:
# Train the classifier.
rf_cls_mdl = ee.Classifier.smileRandomForest(numberOfTrees=100).train(train_data, "class", img_bands)


In [ ]:
# Apply the Classification.
cls_rslts_img = s2_img_percent_dem.classify(rf_cls_mdl)

In [ ]:
map_obj = geemap.Map()

map_obj.centerObject(cls_cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
s2_vis_params = {"min": [150, 100, 187], "max": [4136, 4000, 2712], "bands": ["B8", "B11", "B4"]}

# Add the Sentinel-2 Composite
map_obj.addLayer(s2_med_image, s2_vis_params, "Sentinel-2")

# Specify the visualisation options.
cls_vis_params = {'min': 1, 'max': 3, 'palette': ['green', 'blue', 'grey']}

# Add the Sentinel-2 Classification
map_obj.addLayer(cls_rslts_img, cls_vis_params, "Class")

map_obj

In [ ]:
out_cls_img_file = "n0940e0190_init_dem_cls"
out_gdrive_dir = "mangrove_chng_cls"

# Export the classification image to your Google Drive
task = ee.batch.Export.image.toDrive(
        image=cls_rslts_img,
        description=out_cls_img_file,
        folder=out_gdrive_dir,
        scale=10,
        region=cls_tile_aoi,
        fileFormat="GeoTIFF",
        formatOptions={"cloudOptimized": True, 'noData': 0},
)
task.start()